In [ ]:
import tensorflow as tf
import numpy      as np

import directoryFunctions
import extractFrames
import pathlib
import config
import data
import cv2

from tensorflow.keras.applications.inception_v3 import preprocess_input
from tensorflow.keras.preprocessing             import image
from tensorflow.keras.models                    import load_model, Sequential

"""
Documentation:
-cv2
    1. get(), VideoCapture()
        - https://docs.opencv.org/master/d8/dfe/classcv_1_1VideoCapture.html
    2. imread()
        - https://docs.opencv.org/master/d4/da8/group__imgcodecs.html#ga288b8b3da0892bd651fce07b3bbd3a56
    3. HersheyFonts, LineTypes, putText()
        - https://docs.opencv.org/master/d6/d6e/group__imgproc__draw.html
    2. VideoCapture properties
        - https://docs.opencv.org/master/d4/d15/group__videoio__flags__base.html#gaeb8dd9c89c10a5c63c139bf7c4f5704d
    3. VideoWriter(), write()
        - https://docs.opencv.org/master/dd/d9e/classcv_1_1VideoWriter.html

- numpy
    1. expand_dims()
        - https://numpy.org/doc/stable/reference/generated/numpy.expand_dims.html?highlight=expand_dims#numpy.expand_dims
    2. load()
        - https://numpy.org/doc/stable/reference/generated/numpy.load.html?highlight=load#numpy.load
    3. save()
        - https://numpy.org/doc/stable/reference/generated/numpy.save.html?highlight=save#numpy.save
- pathlib
    1. Path(), /, glob(), .stem
        - https://docs.python.org/3/library/pathlib.html
- tensorflow
    - applications.inception_v3
            1. preprocess_input()
                - https://www.tensorflow.org/versions/r2.1/api_docs/python/tf/keras/applications/inception_v3
    - data.Dataset
        1. predict()
            - https://www.tensorflow.org/versions/r2.1/api_docs/python/tf/data/Dataset
    - keras
        - models
            1. load_model()
                - https://www.tensorflow.org/versions/r2.1/api_docs/python/tf/keras/models/load_model
            2. Sequential()
                1. add(), compile(), fit()
                    - https://www.tensorflow.org/versions/r2.1/api_docs/python/tf/keras/Sequential
        - preprocessing.image
            1. img_to_array(), load_img()
                - https://www.tensorflow.org/versions/r2.1/api_docs/python/tf/keras/preprocessing/image
"""

In [ ]:
"""
Function Name: getPartialModel
Number of parameters: 1
List of parameters:
    1. savedModelPath | str | Path to the saved (CNN) model.
Pre-condition:
    1. savedModelPath exists and is a .h5 file.
Post-condition:
    1. Returns a model that contains some or all of the layers in the model saved in the
       savedModelPath file. If a GlobalAveragePooling2D layer exists, then the returned model will 
       consist of all layers before and including that layer.
"""
def getPartialModel(savedModelPath):
    loadedModel = load_model(savedModelPath)

    indexOfGAPLayer = -1
    # Get the index of the last GlobalAveragePooling2D layer starting from the back of the list of layers
    for layer in loadedModel.layers[::-1]:
        layer_type = str(type(layer))
        if layer_type != "<class 'tensorflow.python.keras.layers.pooling.GlobalAveragePooling2D'>":
            indexOfGAPLayer -= 1
        else:
            break

    if indexOfGAPLayer != -1:
        indexAfterGAPLayer = indexOfGAPLayer + 1
        partialModel = Sequential(loadedModel.layers[:indexAfterGAPLayer])
    else:
        # The GlobalAveragePooling2D Layer is the last layer. Thus, we get all the layers (no need to slice the list).
        partialModel = Sequential(loadedModel.layers[:])

    return partialModel

In [ ]:
"""
Function Name: saveFeatures
Number of parameters: 4
List of parameters:
    1. model           | tf.keras.models | Model that'll be used to extract the features.
    2. framesPath      | pathlib.Path    | Path to the frames whose features are to be extracted.
    3. videoFileName   | str             | Name of the video file that will be used to name the numpy file.
    4. destinationPath | pathlib.Path    | Path used to save the numpy file containing the features.
Pre-condition:
    1. 'framesPath' exists and contains imgages.
    2. 'destinationPath' exists.
Post-condition:
    1. Numpy files each containing an array of features of a frame and is saved at 'destinationPath'.
    2. Nothing is returned.
"""
def saveFeatures(model, framesPath, videoFileName, destinationPath):
    def extractFeatures(model, framePath):
        frame = image.load_img(framePath, 
                               target_size   = (299,299), 
                               interpolation = "lanczos") # shape: (299, 299, 3): # of dim: 3
        frame_arr = image.img_to_array(frame)             # pixel values in range [0,255]
        frame_arr = preprocess_input(frame_arr)           # pixel values in range [-1, 1]
        frame_arr = np.expand_dims(frame_arr, axis = 0)   # expands shape to: (1, 299, 299, 3): # of dim: 4
        features  = model.predict(frame_arr)              # returns numpy array of shape: (1, 2048): # of dim: 2
        features  = features[0]                           # shape: (2048, ): # of dim: 1
        return features
    
    framesPaths = list(sorted(framesPath.glob("*.jpg")))
    numOfDigits = len(str(len(framesPaths)))
    padding     = f'%0{numOfDigits}d'
    for index, framePath in enumerate(framesPaths):
        sequencePath = pathlib.Path(destinationPath/(videoFileName + "_" + str(padding % index)))
        features = extractFeatures(model, framePath)
        features = np.expand_dims(features, axis = 0)
        print(f'Features from Frame # {index} saved at {sequencePath}')
        np.save(sequencePath, features)

In [ ]:
"""
Function Name: predictCNN
Number of parameters: 3
List of parameters:
    1. model      | tf.keras.models | Model that'll be used to make preditions.
    2. framesPath | pathlib.Path    | Path to the frames that'll be used to make predictions.
    3. classNames | list            | List of class names. 
Pre-condition:
    1. 'framesPath' extists.
Post-condition:
    1. Return results after making predictions. The results is a dictionary, where the keys are indicies (int)
       and the values are lists containing a label (str) and a percentage (float).
"""
def predictCNN(model, framesPath, classNames):
    def getPredictions(model, framePath):
        frame = image.load_img(framePath, 
                               target_size   = (299,299), 
                               interpolation = "lanczos")   # shape: (299, 299, 3): # of dim: 3
        frame_arr   = image.img_to_array(frame)             # pixel values in range [0,255]
        frame_arr   = preprocess_input(frame_arr)           # pixel values in range [-1, 1]
        frame_arr   = np.expand_dims(frame_arr, axis = 0)   # expands shape to: (1, 299, 299, 3): # of dim: 4
        predictions = model.predict(frame_arr)
        predictions = predictions[0]
        return predictions
    
    results = {}
    framesPaths = list(sorted(framesPath.glob("*.jpg")))
    for index, framePath in enumerate(framesPaths):
        predictions    = getPredictions(model, framePath)
        topindex       = predictions.argsort()[-1:][0]
        predictedLabel = classNames[topindex]
        percentage     = predictions[topindex] * 100
        results[index] = [predictedLabel, percentage]
    return results

In [ ]:
"""
Function Name: predictRNN
Number of parameters: 3
List of parameters:
    1. model         | tf.keras.models | Model that'll be used to make preditions.
    2. sequencesPath | pathlib.Path    | Path to the sequences that'll be used to make predictions.
    3. classNames    | list            | List of class names. 
Pre-condition:
    1. 'sequencesPath' extists.
Post-condition:
    1. Return results after making predictions. The results is a dictionary, where the keys are indicies (int)
       and the values are lists containing a label (str) and a percentage (float).
"""
def predictRNN(model, sequencesPath, classNames):
    def getPredictions(model, sequencePath):
        sequence      = np.load(sequencePath)
        sequence      = np.expand_dims(sequence, axis = 0)
        predictions   = model.predict(sequence)
        predictions   = predictions[0]
        return predictions
    
    results = {}
    sequencesPaths = list(sorted(sequencesPath.glob("*.npy")))
    for index, sequencePath in enumerate(sequencesPaths):
        predictions    = getPredictions(model, sequencePath)
        topindex       = predictions.argsort()[-1:][0]
        predictedLabel = classNames[topindex]
        percentage     = predictions[topindex] * 100
        results[index] = [predictedLabel, percentage]
    return results

In [ ]:
"""
Function Name: createLabeledVideo
Number of parameters: 5
List of parameters:
    1. videoFilePath  | str          | Path to the video that is to be labeled.
    2. videoFileName  | str          | Name used to name the output video.
    3. outputPath     | pathlib.Path | Path to save the output video.
    4. cnnPredictions | dict         | Dictionary of labels (str) and predictions (int) generated from a CNN Model.
    5. rnnPredictions | dict         | Dictionary of labels (str) and predictions (int) generated from a RNN Model.
Pre-condition:
    1. 'videoFilePath' exists and points to a video.
    2. 'outputPath' exists.
Post-condition:
    1. A video is saved at 'outputPath' that is the labeled version of 'videoFileName'.
    2. Nothing is returned.
"""
def createLabeledVideo(videoFilePath, videoFileName, outputPath, cnnPredictions, rnnPredictions):
    outputFilePath = str(outputPath/f'{videoFileName}_labeled.avi')
    vidCap = cv2.VideoCapture(videoFilePath) # Opens a video file for video capturing
    width  = int(vidCap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(vidCap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fourcc = int(vidCap.get(cv2.CAP_PROP_FOURCC)) # *'XVID'  # fps    = int(vidCap.get(cv2.CAP_PROP_FPS))
    fps    = 8
    
    videoWriter = cv2.VideoWriter(outputFilePath, fourcc, fps, (width, height))
    tempDirectory = pathlib.Path(r"./temp")
    framesPath = tempDirectory/'Frames'
    frames = sorted(framesPath.glob("*"))
    for index, frame in enumerate(frames):
        frame = cv2.imread(str(frame))
        cv2.putText(frame, f'CNN: {cnnPredictions[index][0]} - {round(cnnPredictions[index][1], 2)}%', 
                    (1, 14), cv2.FONT_HERSHEY_DUPLEX , 0.45, (255, 100, 0), 1, cv2.LINE_AA)
        cv2.putText(frame, f'RNN: {rnnPredictions[index][0]} - {round(rnnPredictions[index][1], 2)}%', 
                    (1, 30), cv2.FONT_HERSHEY_DUPLEX, 0.45, (255, 0, 100), 1, cv2.LINE_AA)
        videoWriter.write(frame)
    videoWriter.release()

In [ ]:
"""
Function Name: main
Number of parameters: 0
List of parameters: n/a
Pre-condition: n/a
Post-condition:
    1. A labeled version of a video is saved. Where each frame of the video contains the predicted label and a percentage.
    2. Nothing is returned.
"""
def main():
    dataObj    = data.Data()
    classNames = dataObj.classes
    
    tempDirectory = pathlib.Path(r"./temp")
    directoryFunctions.removeDirectory(tempDirectory)
    directoryFunctions.createDirectory(tempDirectory)
    
    cf            = config.Config()
    rootPath      = pathlib.Path(cf.rootPath)
    ucfVideosPath = pathlib.Path(cf.ucfVideosPath)
    
    # you can insert the path to a video here
    videoFilePath = r"D:\ActionRecognition\UCF-101\Archery\v_Archery_g01_c01.avi"  
    videoFileName = str(pathlib.Path(videoFilePath).stem)
    
    framesPath = tempDirectory/'Frames'
    directoryFunctions.createDirectory(framesPath)
    extractFrames.extractAllFrames(videoFilePath, videoFileName, str(framesPath))
    
    # you can insert the path to a saved model here
    savedCNNModelPath = r"D:\ActionRecognition\Callbacks\CNN\3\ModelCheckpoint\1589823036_CNN_001_0.89.h5" 
    partialCNNModel   = getPartialModel(savedCNNModelPath)
    
    sequencesPath = tempDirectory/'Sequences'
    directoryFunctions.createDirectory(sequencesPath)
    saveFeatures(partialCNNModel, framesPath, videoFileName, sequencesPath)
    
    savedCNNModel  = load_model(savedCNNModelPath)
    cnnPredictions = predictCNN(savedCNNModel, framesPath, classNames)
    
    # you can insert the path to a saved model here
    savedRNNModelPath = r"D:\ActionRecognition\Callbacks\RNN\3\ModelCheckpoint\1589823567_CNN_004_0.70.h5" 
    savedRNNModel     = load_model(savedRNNModelPath)
    rnnPredictions    = predictRNN(savedRNNModel, sequencesPath, classNames)
    
    createLabeledVideo(videoFilePath, videoFileName, tempDirectory, cnnPredictions, rnnPredictions)

In [ ]:
main()